In [1]:
from flask import Flask, render_template, request, redirect, url_for, flash
import psycopg2

In [2]:
app =  Flask(__name__)

## Conexion a Base de Datos

In [3]:
db_host = 'Instancia de Servidor AWS'
db_database = 'Base de Datos'
db_user = 'Usuario de Base de Datos'
db_password = 'Contrasena de Base de Datos'

def connect_postgresql():
    
    conn = psycopg2.connect(
        host=db_host,
        database=db_database,
        user=db_user,
        password=db_password
    )
    
    return conn

## Settings

In [4]:
app.secret_key = 'mysecretkey'

## Funciones de Consulta a Base de Datos

In [5]:
def select_personas():
    query = """
    SELECT idpersona, nombres, apellidopaterno, apellidomaterno, dni, celular, correo
    FROM persona
    """

    conn = connect_postgresql()
    cur = conn.cursor()
    cur.execute(query) 
    
    list_persona = cur.fetchall()

    cur.close()
    conn.close()
    
    return list_persona

def get_person(id):
    query = """
    SELECT idpersona, nombres, apellidopaterno, apellidomaterno, dni, celular, correo
    FROM persona
    WHERE idpersona = {}
    """.format(id)

    conn = connect_postgresql()
    cur = conn.cursor()
    cur.execute(query) 
    
    list_persona = cur.fetchall()

    cur.close()
    conn.close()
    
    return list_persona[0]

## Flask

In [6]:
@app.route('/')

def index():
    
    data = select_personas()

    return render_template('index.html', personas = data)

In [7]:
@app.route('/add_person', methods=['POST'])

def add_person():
    
    try:
        
        if request.method == 'POST':
            nombres = request.form['nombres']
            apellidopaterno = request.form['apellidopaterno']
            apellidomaterno = request.form['apellidomaterno']
            dni = request.form['dni']
            celular = request.form['celular']
            correo = request.form['correo']
        
            query = """INSERT INTO persona (
                            idpersona,
                            nombres,
                            apellidopaterno,
                            apellidomaterno,
                            dni,
                            celular,
                            correo) 
                    VALUES (DEFAULT, '{}', '{}','{}','{}','{}','{}')""".format(nombres,
                                                                               apellidopaterno,
                                                                               apellidomaterno,
                                                                               dni,
                                                                               celular,
                                                                               correo
                                                                              )
            
            # connect to the PostgreSQL server
            conn = connect_postgresql()
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
        
            count = cur.rowcount
        
            message = str(count) + " Record inserted"
        
            cur.close()
            conn.close()
            
            flash(message)

    except Exception as e:
        
        print(e)
        
    return redirect(url_for('index'))

In [8]:
@app.route('/delete_person/<string:id>')

def delete_person(id):
    
    try:
        query = """DELETE FROM persona WHERE idpersona = {}""".format(id)
            
        # connect to the PostgreSQL server
        conn = connect_postgresql()
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        
        count = cur.rowcount
        
        message = str(count) + " Record deleted successfully"
            
        cur.close()
        conn.close()
            
        flash(message)

    except Exception as e:
        
        print(e)
        
    return redirect(url_for('index'))

In [9]:
@app.route('/edit_person/<string:id>')

def edit_person(id):
    data = get_person(id)
    
    return render_template('edit-person.html', persona = data)

In [10]:
@app.route('/update_person/<string:id>', methods = ['POST'])

def update_person(id):
    
    try:
        
        if request.method == 'POST':
            
            nombres = request.form['nombres']
            apellidopaterno = request.form['apellidopaterno']
            apellidomaterno = request.form['apellidomaterno']
            dni = request.form['dni']
            celular = request.form['celular']
            correo = request.form['correo']
            
            query = """UPDATE persona
                       SET nombres = '{}',
                           apellidopaterno = '{}',
                           apellidomaterno = '{}',
                           dni = '{}',
                           celular = '{}',
                           correo = '{}'
                       WHERE idpersona = {}""".format(nombres, apellidopaterno, apellidomaterno, dni, celular, correo, id)
            
            # connect to the PostgreSQL server
            conn = connect_postgresql()
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
        
            count = cur.rowcount
        
            message = str(count) + " Record updated successfully"
            
            cur.close()
            conn.close()
            
            flash(message)

    except Exception as e:
        
        print(e)
        
    return redirect(url_for('index'))

In [ ]:
#Ejecuta el servidor en el puerto 5000
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2021 19:47:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2021 19:47:34] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2021 19:47:34] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2021 19:48:12] "GET /edit_person/2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2021 19:48:18] "POST /update_person/2 HTTP/1.1" 302 -
127.0.0.1 - - [10/Jul/2021 19:48:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2021 20:12:56] "POST /add_person HTTP/1.1" 302 -
127.0.0.1 - - [10/Jul/2021 20:12:57] "GET / HTTP/1.1" 200 -
